In [2]:

import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
import pandas as pd
import datetime

# set the client_ID, client_SECRET, redirect_uri and username for the spotify api authentication
client_ID = '4e1c1626b9e04c0fba6d8f14d31ab3e6'
client_SECRET = '607dd5362f9d4f44b33361eca5aa81b8'   
redirect_uri = 'http://127.0.0.1:9090'
username = 'rt47etgc6xpwhhhb8575rth83'

# data_folder = r"C:\Users\saket\Documents\1.MY_DATA\spotify\spotify api data"
recently_played_file_name = 'recently_played_tracks.csv'

# returns a spotipy object with the given scope
def get_spotify_token(scope):
    client_credentials_manager = spotipy.oauth2.SpotifyOAuth(
        scope=scope,
        username=username,
        client_id=client_ID,
        client_secret=client_SECRET,
        redirect_uri=redirect_uri,
        # open_browser=False
        )
    client_credentials_manager.get_access_token(as_dict=False)
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# returns the recently played tracks
def get_recently_played_tracks(limit=50):
    token = get_spotify_token("user-read-recently-played")
    recently_played = token.current_user_recently_played(limit=limit)
    return recently_played

# returns the top 50 tracks played by the user
def get_top_tracks(limit=50):
    token = get_spotify_token("user-top-read")
    top_tracks = token.current_user_top_tracks(limit=limit)
    return top_tracks

# returns a datetime object from the given timestamp
def timestamp_to_time(timestamp):
    try:
        # Try to parse the timestamp string as ISO format with microsecond component
        dt_object = datetime.datetime.fromisoformat(timestamp[:-1])
    except ValueError:
        try:
            # Try to parse the timestamp string as ISO format without microsecond component
            dt_object = datetime.datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%SZ')
        except ValueError:
            # Try to parse the timestamp string as a Unix timestamp in seconds
            dt_object = datetime.datetime.fromtimestamp(int(timestamp))

    return dt_object

# returns a dictionary with the track info for the given track
def get_track_info_from_json(track_dict):
    # iterate through all the keys in the track_json try to get the value of the key, else set the value to None
    track_info = {}
    for key in track_dict.keys():
        try:
            track_info[key] = track_json[key]
        except:
            track_info[key] = None
    return track_info

# returns a list of dictionaries with the track info for each track
def get_recently_played_tracks_info():
    # list to store the recently played tracks
    recently_played_tracks = []
    # get the recently played tracks
    recently_played = get_recently_played_tracks()
    # iterate through all the recently played tracks
    for i in range(len(recently_played['items'])):
        # get the track info
        track = recently_played['items'][i]['track']
        # create a dictionary to store the track info
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id'],
            'played_at': recently_played['items'][i]['played_at']
        }
        # print the track info
        song_name = track['name']
        # print the track time and name
        # print(recently_played['items'][i]['played_at'], song_name)

        recently_played_tracks.append(track_info)
    return recently_played_tracks




# stores the recently played tracks in a csv file
def store_recently_played_tracks():
    # get the recently played tracks
    recently_played_tracks = get_recently_played_tracks_info()
    # create a dataframe from the stored recently played tracks
    old_recently_played_tracks = pd.read_csv(recently_played_file_name)
    # create a dataframe from the list of recently played tracks
    new_recently_played_tracks = pd.DataFrame(recently_played_tracks)
    # get the last played track from the old_recently_played_tracks
    last_played_at = old_recently_played_tracks['played_at'].iloc[-1]
    # reverse the new_recently_played_tracks
    new_recently_played_tracks = new_recently_played_tracks.iloc[::-1]

    # iterate though all the new_recently_played_tracks and find the one which is closest to the last_played_at
    for i in range(len(new_recently_played_tracks)):
        # get the difference between the last_played_at and the current track's played_at in hours
        diff = (timestamp_to_time(last_played_at) - timestamp_to_time(new_recently_played_tracks['played_at'].iloc[i])).total_seconds() / 3600
        # if the difference is negative, then the current track is the one which was last played
        if diff < 0:
            # get the index of the new_recently_played_tracks
            index = i
            # start the new_recently_played_tracks from the index
            new_recently_played_tracks = new_recently_played_tracks.iloc[index:]
            # iterate through all the new_recently_played_tracks and print the track name and time
            print("Newly added tracks:")
            for j in range(len(new_recently_played_tracks)):
                print(new_recently_played_tracks['played_at'].iloc[j], new_recently_played_tracks['track_name'].iloc[j])
            # add the new_recently_played_tracks to the old_recently_played_tracks, use .concat() to avoid the index being repeated
            old_recently_played_tracks = pd.concat([old_recently_played_tracks, new_recently_played_tracks])
            # save the old_recently_played_tracks to the csv file
            old_recently_played_tracks.to_csv(recently_played_file_name, index=False)
            break

# function to get the spotify token for the given scope
def get_top_tracks(limit=200):
    token = get_spotify_token("user-top-read")
    top_tracks = token.current_user_top_tracks(limit=limit, time_range='short_term')
    return top_tracks

# returns the top tracks info in a list
def get_top_tracks_info():
    top_tracks = get_top_tracks()
    top_tracks_info = []
    for i in range(len(top_tracks['items'])):
        track = top_tracks['items'][i]
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id']
        }
        top_tracks_info.append(track_info)
    return top_tracks_info

# returns the user's top artists in a list
def get_top_artists_info(limit=200):
    token = get_spotify_token("user-top-read")
    top_artists = token.current_user_top_artists(limit=limit)
    top_artists_info = []
    for i in range(len(top_artists['items'])):
        artist = top_artists['items'][i]
        artist_info = {
            'artist_name': artist['name'],
            'artist_id': artist['id'],
            'artist_popularity': artist['popularity'],
            'artist_followers': artist['followers']['total'],
            'artist_genres': artist['genres']
        }
        top_artists_info.append(artist_info)
    return top_artists_info

# returns the user's top genres in a list
def get_top_genres_info(limit=200):
    token = get_spotify_token("user-top-read")
    top_artists = token.current_user_top_artists(limit=limit)
    top_genres_info = []
    for i in range(len(top_artists['items'])):
        artist = top_artists['items'][i]
        for j in range(len(artist['genres'])):
            genre = artist['genres'][j]
            genre_info = {
                'genre': genre
            }
            top_genres_info.append(genre_info)
    return top_genres_info

# returns the user's library tracks in a list
def get_library_info(limit=200):
    # get the token for the user-library-read scope
    token = get_spotify_token("user-library-read")
    # get the user's library data
    library = token.current_user_saved_tracks(limit=limit)

    # convert the library data to a dataframe
    library_df = pd.DataFrame(library)
    return library_df

# returns a list of dictionaries containing the user's playlists
def get_user_playlists():
    token = get_spotify_token("playlist-read-private")
    user_playlists = token.current_user_playlists(limit=50)
    return user_playlists

# returns a list of dictionaries containing the user's playlists information
def get_user_playlists_info():
    user_playlists = get_user_playlists()
    user_playlists_info = []
    for i in range(len(user_playlists['items'])):
        playlist = user_playlists['items'][i]
        playlist_info = {
            'playlist_name': playlist['name'],
            'playlist_id': playlist['id'],
            'playlist_description': playlist['description'],
            'playlist_tracks': playlist['tracks']['total'],
            'playlist_owner': playlist['owner']['display_name'],
            'playlist_owner_id': playlist['owner']['id'],
            'playlist_public': playlist['public'],
            'playlist_collaborative': playlist['collaborative'],
            'playlist_images': playlist['images'][0]['url']
        }
        user_playlists_info.append(playlist_info)
    return user_playlists_info

# stores the user's playlists in a csv file
def store_user_playlists_info():
    user_playlists_info = get_user_playlists_info()
    user_playlists_info_df = pd.DataFrame(user_playlists_info)
    user_playlists_info_df.to_csv('user_playlists_info.csv', index=False)

# returns the tracks in a playlist in a dictionary
def get_user_playlist_tracks(playlist_id):
    token = get_spotify_token("playlist-read-private")
    user_playlist_tracks = token.user_playlist_tracks(playlist_id)
    return user_playlist_tracks

# returns the tracks in a playlist in a list of dictionaries
def get_user_playlist_tracks_info(playlist_id):
    user_playlist_tracks = get_user_playlist_tracks(playlist_id)
    user_playlist_tracks_info = []
    for i in range(len(user_playlist_tracks['items'])):
        track = user_playlist_tracks['items'][i]['track']
        return track
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id']
        }
        user_playlist_tracks_info.append(track_info)
    return user_playlist_tracks_info

# returns a json of recommended tracks according to the seed artists, genres and tracks
def get_recommendations(seed_artists, seed_genres, seed_tracks):
    token = get_spotify_token("user-top-read")
    # returns a json of recommended tracks according to the seed artists, genres and tracks
    recommendations = token.recommendations(seed_artists=seed_artists, seed_genres=seed_genres, seed_tracks=seed_tracks)
    return recommendations

# get important info from recommendations json
def get_recommendations_info(seed_artists, seed_genres, seed_tracks):
    # get the json of the recommended tracks
    recommendations = get_recommendations(seed_artists, seed_genres, seed_tracks)
    recommendations_info = []
    # get the info of each track
    for i in range(len(recommendations['tracks'])):
        track = recommendations['tracks'][i]
        track_info = get_track_info_from_json(track)
        recommendations_info.append(track_info)
    return recommendations_info

# returns the user's saved albums in a list
def get_saved_albums():
    token = get_spotify_token("user-library-read")
    saved_albums = token.current_user_saved_albums()
    return saved_albums

# returns the user's saved shows in a list
def get_saved_shows():
    token = get_spotify_token("user-library-read")
    saved_shows = token.current_user_saved_shows()
    return saved_shows

# returns the user's saved episodes in a list
def get_saved_episodes():
    token = get_spotify_token("user-library-read")
    saved_episodes = token.current_user_saved_episodes()
    return saved_episodes

# returns the user's library tracks in a list
def get_library_info(limit):
    # get the token for the user-library-read scope
    token = get_spotify_token("user-library-read")
    # get the user's library data
    library = token.current_user_saved_tracks(limit=limit)
    # convert the library data to a dataframe
    library_df = pd.DataFrame(library)
    return library_df


# returns a list of dictionaries containing the user's playlists information
def get_user_playlists_info():
    user_playlists = get_user_playlists()
    user_playlists_info = []
    for i in range(len(user_playlists['items'])):
        playlist = user_playlists['items'][i]
        playlist_info = {
            'playlist_name': playlist['name'],
            'playlist_id': playlist['id'],
            'playlist_description': playlist['description'],
            'playlist_tracks': playlist['tracks']['total'],
            'playlist_owner': playlist['owner']['display_name'],
            'playlist_owner_id': playlist['owner']['id'],
            'playlist_public': playlist['public'],
            'playlist_collaborative': playlist['collaborative'],
            'playlist_images': playlist['images'][0]['url']
        }
        user_playlists_info.append(playlist_info)
    return user_playlists_info

# at the top of the df add a raw with current date and time and leave all the other columns empty
# def add_date_row(df):
#     # get the current date and time
#     date = datetime.datetime.now()
#     # convert the date and time to a string
#     date = date.strftime("%Y-%m-%d %H:%M:%S")
#     # create a new row with the date and time
#     new_row = pd.DataFrame({'date': date}, index=[0])
#     # add the new row to the top of the dataframe
#     df = pd.concat([new_row, df]).reset_index(drop=True)
#     return df

def store_user_playlists_info():    
    user_playlists_info = get_user_playlists_info()
    df = pd.DataFrame(user_playlists_info)
    # add a column with the current date and time
    df['date'] = datetime.datetime.now()
    df.to_csv('user_playlists_info.csv', index=False)

# store_user_playlists_info()

user_playlist_info = get_user_playlists_info()
new_df = pd.DataFrame(user_playlist_info)
new_df['date'] = datetime.datetime.now()
# read the csv file
old_df = pd.read_csv('user_playlists_info.csv')




In [25]:
def get_new_playlists(old_df, new_df):
    """Iterate though the new_df and search for matching value from the old dataframe save the index where the first value matches. the value should continue to match."""
    # iterate through the new_df
    the_index = 0
    previous_index = 0
    for i in range(len(new_df)):
        song_id = new_df['playlist_id'][i]
        try:
            # get the index of the matching value
            index = old_df[old_df['playlist_id'] == song_id].index[0]
            # if the index is not the previous index + 1 and it is not the first index
            if i != previous_index+1 and i != 0:
                # save the index
                the_index = i
            previous_index = i
        except IndexError:
            pass
    changed = new_df[:the_index]
    # replace the rows before the index with the new rows, changed
    old_df[:the_index] = changed
    # save the new dataframe to the csv file
    # old_df.to_csv('user_playlists_info.csv', index=False)
    # return all the playlits that are new
    return changed



In [20]:
""" Uesr can add commands in the playlist name or in the playlist discription, this function
searches for the all the possible commands and returns a list of the commands """
def get_commands(dataframe):
    # get the playlist name and description
    playlist_name = dataframe['playlist_name']
    playlist_description = dataframe['playlist_description']
    # List of commands to arrange the playlists
    rearrange = {   'most played' : ["top", "most played", "top played"],
                    'least played' : ["least played"],
                    'most recently played' : ["most recently played", "most recent", "recently played", "recent"],
                    'recently played' : ["recently played", "recent", "last played"],
                 }
    # command meaning
    key_meaning = { 'artist' : ['artist', 'a'],
            'album' : ['album', 'al'],
            'song' : ['song', 's'],
            'playlist' : ['playlist', 'p'],
            'genre' : ['genre', 'g'],
            'find' : ['find', 'f', 'search', 's'],
            'generate' : ['generate', 'g'],
    }
    # create a list of the commands that are in the playlist name
    name_commands = []
    # create a list of the commands that are in the playlist description
    description_commands = []

# arrage the songs that were played 
def arrange_top_played(playlist_id, days=50):


,playlist_name,playlist_id,playlist_description,playlist_tracks,playlist_owner,playlist_owner_id,playlist_public,playlist_collaborative,playlist_images,date
0,Human Sadness Radio,37i9dQZF1E8JhGUxJHmvnq,,50,Spotify,spotify,False,False,https://seeded-session-images.scdn.co/v1/img/t...,2023-03-26 16:10:34.986707
1,Poorvi + Saket,37i9dQZF1EJLsvVumBJwJe,A blend of music for Saket and Poorvi. Updates...,50,Spotify,spotify,False,False,https://blend-playlist-covers.spotifycdn.com/v...,2023-03-26 16:10:34.986707
2,Best of Radiohead,7sb69mB0VFyz4t1PwkFGMC,,13,Saket,rt47etgc6xpwhhhb8575rth83,True,False,https://mosaic.scdn.co/640/ab67616d0000b2736c7...,2023-03-26 16:10:34.986707
3,You and me 🌈🌼🌸,6wM3aS0HC00gw7R8XhzbO5,,308,Poorvi,l57o12u24498uhn083lff5vy3,False,False,https://mosaic.scdn.co/640/ab67616d0000b27350d...,2023-03-26 16:10:34.986707


In [29]:
import requests
import json

# Set the target word and the context
word = "apple"
context = "general"

# Set the Owl API endpoint
url = "https://owlapi.herokuapp.com/api/v1/similar"

# Set the parameters for the request
params = {
    "word": word,
    "context": context
}

# Send a GET request to the Owl API
response = requests.get(url, params=params)

# Parse the JSON response
data = response.json()

# Print the most similar words
print("The most similar words to", word, "in", context, "context are:")
for item in data["similar_words"]:
    print(item["word"], "-", item["similarity"])

In [69]:

# # a functions that finds what difference were made in the new dataframe and the old one
# def find_difference(new_df, old_df):
    


In [46]:
tracks_info = get_recommendations_info(["79S80ZWgVhIPMCHuvl6SkA"], ["post-rock"], ["05JqOBN6XW4eFUVQlgR0I3"])


In [ ]:

# playlist-read-collaborative: Include collaborative playlists when requesting a user’s playlists1.
# playlist-modify-public: Write access to a user’s public playlists1.
# playlist-modify-private: Write access to a user’s private playlists1.


In [13]:
# dictionaries containing the user's playlists information
library_tracks_info = get_library_info(limit=50)

In [18]:
# get all the user's playlists
user_playlists = get_user_playlists_info()

href
items
limit
next
offset
previous
total


In [49]:
# iterate through the first track from tracks_info and print all the keys
for key in tracks_info[0]:
    print(key)
    


album
artists
available_markets
disc_number
duration_ms
explicit
external_ids
external_urls
href
id
is_local
name
popularity
preview_url
track_number
type
uri


In [4]:

# store the recently played tracks
store_recently_played_tracks()




In [26]:
# get the uers's playlists
user_playlists = get_user_playlists(get_spotify_token("playlist-read-private"))

In [28]:
for i in user_playlists:
    print(i)

href
items
limit
next
offset
previous
total


In [13]:
# function read the most recently made playlist
def read_most_recent_playlist():
    playlists_list = get_user_playlists_info()
    # arrange the playlists by date
    playlists_list.sort(key=lambda x: x['playlist_id'], reverse=True)




In [102]:
# location of the folder to store the data



# # get the top 50 tracks played by the user
# def get_top_tracks(token):
#     top_tracks = token.current_user_top_tracks(limit=50)
#     return top_tracks


In [4]:
data_folder = r"C:\Users\saket\Documents\1.MY_DATA\spotify\spotify api data"
recently_played_file_name = 'recently_played_tracks.csv'

# function to add the recently played tracks in a csv file
def store_recently_played_tracks():
    recently_played_tracks = get_recently_played_tracks_info(recently_played)
    # create a dataframe from the list of recently played tracks
    recently_played_file_path = os.path.join(data_folder, recently_played_file_name)
    old_recently_played_tracks = pd.read_csv(recently_played_file_path)
    new_recently_played_tracks = pd.DataFrame(recently_played_tracks)
    # get the last played track from the old_recently_played_tracks
    last_played_at = old_recently_played_tracks['played_at'].iloc[-1]
    # reverse the new_recently_played_tracks
    new_recently_played_tracks = new_recently_played_tracks.iloc[::-1]

    # iterate though all the new_recently_played_tracks and find the one which is closest to the last_played_at
    for i in range(len(new_recently_played_tracks)):
        # get the difference between the last_played_at and the current track's played_at in hours
        diff = (timestamp_to_time(last_played_at) - timestamp_to_time(new_recently_played_tracks['played_at'].iloc[i])).total_seconds() / 3600
        # print(diff, new_recently_played_tracks['track_name'].iloc[i])
        # if the difference is negative, then the current track is the one which was last played
        if diff < 0:
            # get the index of the new_recently_played_tracks
            index = i
            # start the new_recently_played_tracks from the index
            new_recently_played_tracks = new_recently_played_tracks.iloc[index:]
            # print(new_recently_played_tracks['track_name'].iloc[index])
                # add the new_recently_played_tracks to the old_recently_played_tracks, use .concat() to avoid the index being repeated
            old_recently_played_tracks = pd.concat([old_recently_played_tracks, new_recently_played_tracks])
            # save the old_recently_played_tracks to the csv file
            old_recently_played_tracks.to_csv(recently_played_file_path, index=False)
            break

store_recently_played_tracks()



NameError: name 'recently_played' is not defined

In [ ]:
# import datetime
import datetime

# get all the useful information from the recently played tracks
def get_recently_played_tracks_info(recently_played):
    recently_played_tracks = []
    recently_played_token = get_spotify_token("user-read-recently-played")
    recently_played = get_recently_played_tracks(recently_played_token)
    for i in range(len(recently_played['items'])):
        track = recently_played['items'][i]['track']
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id'],
            'played_at': recently_played['items'][i]['played_at']
        }
        # song name
        song_name = track['name']
        print(recently_played['items'][i]['played_at'], song_name)



        recently_played_tracks.append(track_info)
    return recently_played_tracks

# def store_recently_played_tracks():
    recently_played_tracks = get_recently_played_tracks_info(recently_played)
    # create a dataframe from the list of recently played tracks
    # recently_played_file_path = os.path.join(data_folder, recently_played_file_name)
    old_recently_played_tracks = pd.read_csv(recently_played_file_name)
    new_recently_played_tracks = pd.DataFrame(recently_played_tracks)
    # get the last played track from the old_recently_played_tracks
    last_played_at = old_recently_played_tracks['played_at'].iloc[-1]
    # reverse the new_recently_played_tracks
    new_recently_played_tracks = new_recently_played_tracks.iloc[::-1]

    # iterate though all the new_recently_played_tracks and find the one which is closest to the last_played_at
    for i in range(len(new_recently_played_tracks)):
        # get the difference between the last_played_at and the current track's played_at in hours
        diff = (timestamp_to_time(last_played_at) - timestamp_to_time(new_recently_played_tracks['played_at'].iloc[i])).total_seconds() / 3600
        # print(diff, new_recently_played_tracks['track_name'].iloc[i])
        # if the difference is negative, then the current track is the one which was last played
        if diff < 0:
            # get the index of the new_recently_played_tracks
            index = i
            # start the new_recently_played_tracks from the index
            new_recently_played_tracks = new_recently_played_tracks.iloc[index:]
            # print(new_recently_played_tracks['track_name'].iloc[index])
                # add the new_recently_played_tracks to the old_recently_played_tracks, use .concat() to avoid the index being repeated
            old_recently_played_tracks = pd.concat([old_recently_played_tracks, new_recently_played_tracks])
            # save the old_recently_played_tracks to the csv file
            old_recently_played_tracks.to_csv(recently_played_file_path, index=False)
            break


    # # iterate through all the library tracks
    # for i in range(len(library['items'])):
    #     # get the track
    #     track = library['items'][i]['track']
    #     # get the track info
    #     track_info = {
    #         'track_name': track['name'],
    #         'track_id': track['id'],
    #         'track_duration': track['duration_ms'],
    #         'track_popularity': track['popularity'],
    #         'track_explicit': track['explicit'],
    #         'album_name': track['album']['name'],
    #         'album_id': track['album']['id'],
    #         'album_release_date': track['album']['release_date'],
    #         'album_release_date_precision': track['album']['release_date_precision'],
    #         'album_total_tracks': track['album']['total_tracks'],
    #         'album_type': track['album']['album_type'],
    #         'artist_name': track['artists'][0]['name'],
    #         'artist_id': track['artists'][0]['id']
    #     }
    #     # add the track info to the library info
    #     library_info.append(track_info)


5


In [ ]:
time.time()

1678126686.844148

In [ ]:
# iterate through the first history item and print the track name
for item in history['items']:
    track = item['track']
    print(track['name'])

Oxygen
Hunter
Toxicity
Mother of the World
Screen Shot
Lunacy
The Blacker The Berry
Messenger
Simulation Swarm
Hey Jane
Song Against Sex
She Loves Us
Hacker
She Loves Us
Nosferatu Man
Oxygen
Mladic
Mladic
Mladic
IGOR'S THEME
Flume
Punkero Sonidero
money machine
1539 N. Calvert
Helplessness Blues
Mladic
The Black Hawk War, Or, How to Demolish an Entire Civilization and Still Feel Good About Yourself In the Morning, Or, We Apologize for the Inconvenience But You're Going to Have to Leave Now, Or...
Concerning the UFO sighting near Highland, Illinois
King of Carrot Flowers Pts. 2 & 3
A short reprise for Mary Todd, who went insane, but for very good reasons
Jacksonville
John Wayne Gacy, Jr.
Come On! Feel the Illinoise! Part I: The World's Columbian Exposition Part II: Carl Sandburg Visits Me In A Dream
The Black Hawk War, Or, How to Demolish an Entire Civilization and Still Feel Good About Yourself In the Morning, Or, We Apologize for the Inconvenience But You're Going to Have to Leave Now